# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [20]:
driver = webdriver.Chrome()

In [21]:
driver.get('https://www.tdlr.texas.gov/tools_search/')

In [22]:
fill_in = driver.find_element_by_id('mcrdata')
fill_in.send_keys('006179570C')

In [23]:
button = driver.find_element_by_id('submit3')
button.click()

In [24]:
company = driver.find_elements_by_tag_name('td')[5].text.split('   ')[1]
company
owner = driver.find_elements_by_tag_name('td')[7].text.split('   ')[1]
owner
phone_number = driver.find_elements_by_tag_name('td')[9].text.split('   ')[1]
phone_number
status = driver.find_elements_by_tag_name('td')[12].text.split('  ')[1]
status
address = driver.find_elements_by_tag_name('td')[14].text.split('  ')[2].split(':')[2].replace('\n', '')
address

'13619 BRETT JACKSON RD.FORT WORTH, TX. 76179'

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [25]:
# Go to webpage
driver.get('https://www.tdlr.texas.gov/tools_search/')

# Fill in and send the form
fill_in = driver.find_element_by_id('mcrdata')
fill_in.send_keys('006179570C')
button = driver.find_element_by_id('submit3')
button.click()


# Scrape and print all the stuff

company = driver.find_elements_by_tag_name('td')[5].text.split('   ')[1]
print(company)

owner = driver.find_elements_by_tag_name('td')[7].text.split('   ')[1]
print(owner)

phone_number = driver.find_elements_by_tag_name('td')[9].text.split('   ')[1]
print(phone_number)

status = driver.find_elements_by_tag_name('td')[12].text.split('  ')[1]
print(status)

address = driver.find_elements_by_tag_name('td')[14].text.split('  ')[2].split(':')[2].replace('\n', '')
print(address)


B.D. SMITH TOWING
BRANDT SMITH / OWNER
8173330706
Expired
13619 BRETT JACKSON RD.FORT WORTH, TX. 76179


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [26]:
df_subset = pd.read_csv('trucks-subset.csv')

In [27]:
df_subset

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [28]:
def get_url(TDLR):
    url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=' + TDLR
    return url

In [29]:
pd.set_option('display.max_colwidth', -1)
df_subset['TDLR Number'].apply(get_url)

0    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
Name: TDLR Number, dtype: object

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [30]:
df_subset['url'] = df_subset['TDLR Number'].apply(get_url)

In [31]:
df_subset

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [47]:
def get_information(row):
    
    driver.get(row['url'])
    
    company = driver.find_elements_by_tag_name('td')[5].text
    print(company)

    owner = driver.find_elements_by_tag_name('td')[7].text
    print(owner)

    phone_number = driver.find_elements_by_tag_name('td')[9].text
    print(phone_number)

    status = driver.find_elements_by_tag_name('td')[12].text
    print(status)

    address = driver.find_elements_by_tag_name('td')[14].text
    print(address)
    
    return pd.Series({
        'company': company,
        'owner': owner,
        'phone_number': phone_number,
        'status': status,
        'address': address
    })

In [33]:
df_subset.apply(get_information, axis = 1)

AUGUSTUS E SMITH
AUGUSTUS EUGENE SMITH / OWNER
9032276464
Active
103 N MAIN STBONHAM, TX. 75418
AUGUSTUS E SMITH
AUGUSTUS EUGENE SMITH / OWNER
9032276464
Active
103 N MAIN STBONHAM, TX. 75418
B.D. SMITH TOWING
BRANDT SMITH / OWNER
8173330706
Expired
13619 BRETT JACKSON RD.FORT WORTH, TX. 76179
BARRY MICHAEL SMITH
BARRY MICHAEL SMITH / OWNER
8066544404
Active
4501 W CEMETERY RDCANYON, TX. 79015


,company,owner,phone_number,status,address
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN STBONHAM, TX. 75418"
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD.FORT WORTH, TX. 76179"
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RDCANYON, TX. 79015"


## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [34]:
df_subset_extended = df_subset.apply(get_information, axis = 1).join(df_subset)

AUGUSTUS E SMITH
AUGUSTUS EUGENE SMITH / OWNER
9032276464
Active
103 N MAIN STBONHAM, TX. 75418
AUGUSTUS E SMITH
AUGUSTUS EUGENE SMITH / OWNER
9032276464
Active
103 N MAIN STBONHAM, TX. 75418
B.D. SMITH TOWING
BRANDT SMITH / OWNER
8173330706
Expired
13619 BRETT JACKSON RD.FORT WORTH, TX. 76179
BARRY MICHAEL SMITH
BARRY MICHAEL SMITH / OWNER
8066544404
Active
4501 W CEMETERY RDCANYON, TX. 79015


### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [35]:
df_subset_extended.to_csv('tow-trucks-extended.csv', index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [36]:
df_subset_extended = pd.read_csv('tow-trucks-extended.csv')
df_subset_extended

,company,owner,phone_number,status,address,TDLR Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN STBONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD.FORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RDCANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [37]:
df = pd.read_csv('tow-trucks.csv')

In [38]:
df['url'] = df['TDLR Number'].apply(get_url)
df

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,006495492C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C


## GOING TO REDEFINE THE FUNCTION FOR DEBUGGING PURPOSES

In [57]:
def get_information(row):
    
    driver.get(row['url'])
    
    company = driver.find_elements_by_tag_name('td')[5].text

    owner = driver.find_elements_by_tag_name('td')[7].text

    phone_number = driver.find_elements_by_tag_name('td')[9].text
    
    # Some vehicles had two owners which was messing with my scraping, so i used this conditional to fix that up
    
    if phone_number[:1] == 'O':
        owner = [owner, phone_number]
        phone_number = driver.find_elements_by_tag_name('td')[11].text
        status = driver.find_elements_by_tag_name('td')[14].text
        address = driver.find_elements_by_tag_name('td')[16].text
    else:
        status = driver.find_elements_by_tag_name('td')[12].text

        address = driver.find_elements_by_tag_name('td')[14].text
    
    return pd.Series({
        'company': company,
        'owner': owner,
        'phone_number': phone_number,
        'status': status,
        'address': address
    })

In [58]:
df_extended = df.apply(get_information, axis=1).join(df)

In [59]:
df_extended

,company,owner,phone_number,status,address,TDLR Number,url
0,Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Status: Active,"Carrier Type: Tow Truck Company\nNumber of Active Tow Trucks: 1\n\nAddress Information\nMailing:\n1617 COUNTY ROAD 4130\nBONHAM, TX. 75418\n\nPhysical:\n103 N MAIN ST\nBONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Status: Expired,"Carrier Type: Tow Truck Company\nNumber of Active Tow Trucks: 0\n\nAddress Information\nMailing:\n13619 BRETT JACKSON RD\nFORT WORTH, TX. 76179\n\nPhysical:\n13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Status: Active,"Carrier Type: Tow Truck Company\nNumber of Active Tow Trucks: 4\n\nAddress Information\nMailing:\n4501 W CEMETERY RD\nCANYON, TX. 79015\n\nPhysical:\n4501 W CEMETERY RD\nCANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,Name: HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 940-552-0687,Status: Expired,"Carrier Type: Tow Truck Company\nNumber of Active Tow Trucks: 0\n\nAddress Information\nMailing:\n1529 WILBARGER ST\nVERNON, TX. 76384\n\nPhysical:\n1529 WILBARGER ST\nVERNON, TX. 76384",006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,Name: HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 9405520687,Status: Expired,"Carrier Type: Vehicle Storage Facility\nCapacity: <=50\n\nAddress Information\nMailing:\n1529 WILBARGER ST\nVERNON, TX. 76384\n\nPhysical:\n1529 WILBARGER ST\nVERNON, TX. 76384",0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,Name: HYSMITH AUTOMOTIVE,"[Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT, Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER]",Phone: 940-521-0294,Status: Active,"Carrier Type: Tow Truck Company\nNumber of Active Tow Trucks: 7\n\nAddress Information\nMailing:\n1210 US 380 BYPASS\nGRAHAM, TX. 76450\n\nPhysical:\n1210 US 380 BYPASS\nGRAHAM, TX. 76450",006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,Name: HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,"[Owner/Officer: HYSMITH ERIN ASHLEY / TREASURER, Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT]",Phone: 9405210294,Status: Suspended,"Carrier Type: Vehicle Storage Facility\nCapacity: <=50\n\nAddress Information\nMailing:\n927 LOVING HWY\nGRAHAM, TX. 76450\n\nPhysical:\n927 LOVING HWY\nGRAHAM, TX. 76450",0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,Name: HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,"[Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT, Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER]",Phone: 9405210294,Status: Active,"Carrier Type: Vehicle Storage Facility\nCapacity: <=50\n\nAddress Information\nMailing:\n1210 380 BYPASS\nGRAHAM, TX. 76450\n\nPhysical:\n1210 380 BYPASS\nGRAHAM, TX. 76450",0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,Name: JEFF & WENDY SMITH,"[Owner/Officer: JEFF SMITH / PARTNER, Owner/Officer: WENDY SMITH / PARTNER]",Phone: 903 586 6234,Status: Suspended,"Carrier Type: Tow Truck Company\nNumber of Active Tow Trucks: 0\n\nAddress Information\nMailing:\n10842 FM 2138 N\nJACKSONVILLE, TX. 75766\n\nPhysical:\n10842 FM 2138 N\nJACKSONVILLE, TX. 75766",006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,Name: JEFF SMITH,Owner/Officer: JEFFREY JOHN SMITH / OWNER,Phone: 8324354670,Status: Active,"Carrier Type: Tow Truck Company\nNumber of Active Tow Trucks: 1\n\nAddress Information\nMailing:\n4338 HARVEY RD\nCROSBY, TX. 77532\n\nPhysical:\n4338 HARVEY RD\nCROSBY, TX. 77532",006495492C,https://

In [60]:
df_extended.to_csv('tow-trucks-extended.csv', index=False)

In [61]:
df_final = pd.read_csv('tow-trucks-extended.csv')
df_final

,company,owner,phone_number,status,address,TDLR Number,url
0,Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Status: Active,"Carrier Type: Tow Truck Company\nNumber of Active Tow Trucks: 1\n\nAddress Information\nMailing:\n1617 COUNTY ROAD 4130\nBONHAM, TX. 75418\n\nPhysical:\n103 N MAIN ST\nBONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Status: Expired,"Carrier Type: Tow Truck Company\nNumber of Active Tow Trucks: 0\n\nAddress Information\nMailing:\n13619 BRETT JACKSON RD\nFORT WORTH, TX. 76179\n\nPhysical:\n13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Status: Active,"Carrier Type: Tow Truck Company\nNumber of Active Tow Trucks: 4\n\nAddress Information\nMailing:\n4501 W CEMETERY RD\nCANYON, TX. 79015\n\nPhysical:\n4501 W CEMETERY RD\nCANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,Name: HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 940-552-0687,Status: Expired,"Carrier Type: Tow Truck Company\nNumber of Active Tow Trucks: 0\n\nAddress Information\nMailing:\n1529 WILBARGER ST\nVERNON, TX. 76384\n\nPhysical:\n1529 WILBARGER ST\nVERNON, TX. 76384",006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,Name: HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 9405520687,Status: Expired,"Carrier Type: Vehicle Storage Facility\nCapacity: <=50\n\nAddress Information\nMailing:\n1529 WILBARGER ST\nVERNON, TX. 76384\n\nPhysical:\n1529 WILBARGER ST\nVERNON, TX. 76384",0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,Name: HYSMITH AUTOMOTIVE,"['Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT', 'Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER']",Phone: 940-521-0294,Status: Active,"Carrier Type: Tow Truck Company\nNumber of Active Tow Trucks: 7\n\nAddress Information\nMailing:\n1210 US 380 BYPASS\nGRAHAM, TX. 76450\n\nPhysical:\n1210 US 380 BYPASS\nGRAHAM, TX. 76450",006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,Name: HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,"['Owner/Officer: HYSMITH ERIN ASHLEY / TREASURER', 'Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT']",Phone: 9405210294,Status: Suspended,"Carrier Type: Vehicle Storage Facility\nCapacity: <=50\n\nAddress Information\nMailing:\n927 LOVING HWY\nGRAHAM, TX. 76450\n\nPhysical:\n927 LOVING HWY\nGRAHAM, TX. 76450",0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,Name: HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,"['Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT', 'Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER']",Phone: 9405210294,Status: Active,"Carrier Type: Vehicle Storage Facility\nCapacity: <=50\n\nAddress Information\nMailing:\n1210 380 BYPASS\nGRAHAM, TX. 76450\n\nPhysical:\n1210 380 BYPASS\nGRAHAM, TX. 76450",0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,Name: JEFF & WENDY SMITH,"['Owner/Officer: JEFF SMITH / PARTNER', 'Owner/Officer: WENDY SMITH / PARTNER']",Phone: 903 586 6234,Status: Suspended,"Carrier Type: Tow Truck Company\nNumber of Active Tow Trucks: 0\n\nAddress Information\nMailing:\n10842 FM 2138 N\nJACKSONVILLE, TX. 75766\n\nPhysical:\n10842 FM 2138 N\nJACKSONVILLE, TX. 75766",006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,Name: JEFF SMITH,Owner/Officer: JEFFREY JOHN SMITH / OWNER,Phone: 8324354670,Status: Active,"Carrier Type: Tow Truck Company\nNumber of Active Tow Trucks: 1\n\nAddress Information\nMailing:\n4338 HARVEY RD\nCROSBY, TX. 77532\n\nPhysical:\n4338 HARVEY RD\nCROSBY, TX. 77532",006